In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from ATLAS_style import *
ROOT.gErrorIgnoreLevel = ROOT.kError
set_atlas_style()

In module 'Darwin':
/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/libkern/arm/OSByteOrder.h:14:1: error: '_OSSwapInt16' has different definitions in different modules; definition in module 'Darwin.libkern.OSByteOrder' first difference is return type is 'uint16_t' (aka 'unsigned short')
uint16_t
^~~~~~~~
/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/libkern/arm/_OSByteOrder.h:49:1: note: but in 'DarwinFoundation.OSByteOrder' found different return type '__uint16_t' (aka 'unsigned short')
__uint16_t
^~~~~~~~~~
In module 'Darwin':
/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/libkern/arm/OSByteOrder.h:24:1: error: '_OSSwapInt32' has different definitions in different modules; definition in module 'Darwin.libkern.OSByteOrder' first difference is return type is 'uint32_t' (aka 'unsigned int')
uint32_t
^~~~~~~~


Welcome to JupyROOT 6.30/02
Applying ATLAS style settings


## plot distributions and efficiencies

In [2]:
file_HIST_z = ROOT.TFile.Open("dec20/root_z/HIST.root")
file_HIST_gamma = ROOT.TFile.Open("dec20/root_gamma/HIST.root")

In [3]:
base_dir = 'run_470000/HLT/TauMon/HLT_Efficiency'
file_base_dir = file_HIST_z.Get(base_dir)
HLTpath_dir = []

keys = file_base_dir.GetListOfKeys()
for key in keys:
    obj = key.ReadObj()
    if obj.InheritsFrom("TDirectory"):
        HLTpath_dir.append(obj.GetName())

HLTpath_subdir = ['HLT_Efficiency_1P', 'HLT_Efficiency_3P']
HLTpath_eff = ['EffHLT_averageMu_wrt_Offline',
               'EffHLT_tauEta_highPt_wrt_Offline', 'EffHLT_tauEta_wrt_Offline',
               'EffHLT_tauPhi_highPt_wrt_Offline', 'EffHLT_tauPhi_wrt_Offline',
               'EffHLT_tauPt_coarse_wrt_Offline', 'EffHLT_tauPt_wrt_Offline',
              ]

In [6]:
def plot_eff_single(path, eff, label, out_dir, ymin=0, ymax=1.6):    
    canvas = ROOT.TCanvas(path, "", 1000, 800)
    
    eff.SetLineColor(ROOT.kBlack)
    
    eff.Draw("AP")

    canvas.Update()

    graph = eff.GetPaintedGraph()
    if graph:
        graph.GetYaxis().SetRangeUser(ymin, ymax)

    canvas.Modified()
    canvas.Update()

    # 'run_470000/HLT/TauMon/HLT_Efficiency/HLT_tau80_mediumRNN_tracktwoMVA_L1eTAU80/HLT_Efficiency_3P/EffHLT_tauPt_wrt_Offline'
    
    path_parts = path.split('/')
    prong = 'NA'
    if '1P' in path_parts[5]:
        prong = '1P'
    elif '3P' in path_parts[5]:
        prong = '3P'
    
    legend = ROOT.TLegend(0.2, 0.87, 0.45, 0.7)
    legend.SetHeader(path_parts[4] + ' ' + prong + ' ' + path_parts[6])
    
    header_entry = legend.GetListOfPrimitives().At(0)
    header_entry.SetTextSize(20)
    legend.SetTextSize(20)
    legend.AddEntry(eff, label, "lp")
    legend.Draw()

    canvas.Update()
    ATLASlabel(0.2, 0.88, "Internal")

    save_dir = out_dir + path_parts[4] + '/' + prong
    if not os.path.exists(save_dir):
        !mkdir -p {save_dir}
    canvas.SaveAs(save_dir + '/' + path_parts[6] + '.png')

    del canvas, legend

for HLT_path in HLTpath_dir:
    for prong in HLTpath_subdir:
        for eff in HLTpath_eff:
            eff_path = base_dir + '/' + HLT_path + '/' + prong + '/' + eff
            plot_eff_single(path=eff_path,
                    eff=file_HIST_z.Get(eff_path),
                    label='tau.mc23_13p6TeV.601191.PhPy8EG_AZNLO_Ztautau',
                    out_dir='dec20/eff/z/',
                    ymin=0,
                    ymax=1.6
                    )
            
for HLT_path in HLTpath_dir:
    for prong in HLTpath_subdir:
        for eff in HLTpath_eff:
            eff_path = base_dir + '/' + HLT_path + '/' + prong + '/' + eff
            plot_eff_single(path=eff_path,
                    eff=file_HIST_gamma.Get(eff_path),
                    label='tau.mc23_13p6TeV.801002.Py8EG_A14NNPDF23LO_Gammatautau_MassWeight',
                    out_dir='dec20/eff/gamma/',
                    ymin=0,
                    ymax=1.6
                    )

In [7]:
def compare_eff(path, eff_1, eff_2, label_1, label_2, out_dir ,ymin=0, ymax=1.6):    
    canvas = ROOT.TCanvas(path, "", 1000, 800)
    
    eff_1.SetLineColor(ROOT.kBlack)
    eff_2.SetLineColor(ROOT.kRed)
    
    eff_1.Draw("AP")
    eff_2.Draw("P SAME")

    canvas.Update()

    graph_1 = eff_1.GetPaintedGraph()
    if graph_1:
        graph_1.GetYaxis().SetRangeUser(ymin, ymax)

    canvas.Modified()
    canvas.Update()

    # 'run_470000/HLT/TauMon/HLT_Efficiency/HLT_tau80_mediumRNN_tracktwoMVA_L1eTAU80/HLT_Efficiency_3P/EffHLT_tauPt_wrt_Offline'
    
    path_parts = path.split('/')
    prong = 'NA'
    if '1P' in path_parts[5]:
        prong = '1P'
    elif '3P' in path_parts[5]:
        prong = '3P'
    
    legend = ROOT.TLegend(0.2, 0.87, 0.45, 0.7)
    legend.SetHeader(path_parts[4] + ' ' + prong + ' ' + path_parts[6])
    
    header_entry = legend.GetListOfPrimitives().At(0)
    header_entry.SetTextSize(20)
    legend.SetTextSize(20)
    legend.AddEntry(eff_1, label_1, "lp")
    legend.AddEntry(eff_2, label_2, "lp")
    legend.Draw()

    canvas.Update()
    ATLASlabel(0.2, 0.88, "Internal")

    save_dir = out_dir + path_parts[4] + '/' + prong
    if not os.path.exists(save_dir):
        !mkdir -p {save_dir}
    canvas.SaveAs(save_dir + '/' + path_parts[6] + '.png')

    del canvas, legend

for HLT_path in HLTpath_dir:
    for prong in HLTpath_subdir:
        for eff in HLTpath_eff:
            eff_path = base_dir + '/' + HLT_path + '/' + prong + '/' + eff
            compare_eff(path=eff_path,
                        eff_1=file_HIST_z.Get(eff_path),
                        eff_2=file_HIST_gamma.Get(eff_path),
                        label_1='tau.mc23_13p6TeV.601191.PhPy8EG_AZNLO_Ztautau',
                        label_2='tau.mc23_13p6TeV.801002.Py8EG_A14NNPDF23LO_Gammatautau_MassWeight',
                        out_dir='dec20/eff/compare/',
                        ymin=0,
                        ymax=1.6
                       )

## compare releases

In [ ]:
file_HIST_ref = ROOT.TFile.Open("release_68/out.HIST.root")
file_HIST_new = ROOT.TFile.Open("release_72/out.HIST.root")

In [ ]:
base_dir = 'run_470000/HLT/TauMon/HLT_Efficiency'
file_base_dir = file_HIST_ref.Get(base_dir)
HLTpath_dir = []

keys = file_base_dir.GetListOfKeys()
for key in keys:
    obj = key.ReadObj()
    if obj.InheritsFrom("TDirectory"):
        HLTpath_dir.append(obj.GetName())

HLTpath_subdir = ['HLT_Efficiency_1P', 'HLT_Efficiency_3P']
HLTpath_eff = ['EffHLT_averageMu_wrt_Offline',
               'EffHLT_tauEta_highPt_wrt_Offline', 'EffHLT_tauEta_wrt_Offline',
               'EffHLT_tauPhi_highPt_wrt_Offline', 'EffHLT_tauPhi_wrt_Offline',
               'EffHLT_tauPt_coarse_wrt_Offline', 'EffHLT_tauPt_wrt_Offline',
              ]

In [ ]:
def compare_eff(path, eff_ref, eff_new, release_ref, release_new, ymin=0, ymax=1.6):    
    canvas = ROOT.TCanvas(path, "", 1000, 800)
    
    eff_ref.SetLineColor(ROOT.kBlack)
    eff_new.SetLineColor(ROOT.kRed)
    
    eff_ref.Draw("AP")
    eff_new.Draw("P SAME")

    canvas.Update()

    graph_ref = eff_ref.GetPaintedGraph()
    if graph_ref:
        graph_ref.GetYaxis().SetRangeUser(ymin, ymax)

    canvas.Modified()
    canvas.Update()

    # 'run_470000/HLT/TauMon/HLT_Efficiency/HLT_tau80_mediumRNN_tracktwoMVA_L1eTAU80/HLT_Efficiency_3P/EffHLT_tauPt_wrt_Offline'
    
    path_parts = path.split('/')
    prong = 'NA'
    if '1P' in path_parts[5]:
        prong = '1P'
    elif '3P' in path_parts[5]:
        prong = '3P'
    
    legend = ROOT.TLegend(0.2, 0.87, 0.45, 0.7)
    legend.SetHeader(path_parts[4] + ' ' + prong + ' ' + path_parts[6])
    
    header_entry = legend.GetListOfPrimitives().At(0)
    header_entry.SetTextSize(20)
    legend.SetTextSize(20)
    legend.AddEntry(eff_ref, release_ref, "lp")
    legend.AddEntry(eff_new, release_new, "lp")
    legend.Draw()

    canvas.Update()
    ATLASlabel(0.2, 0.88, "Internal")

    save_dir = 'plots/' + path_parts[4] + '/' + prong
    if not os.path.exists(save_dir):
        !mkdir -p {save_dir}
    canvas.SaveAs(save_dir + '/' + path_parts[6] + '.png')

    del canvas, legend

for HLT_path in HLTpath_dir:
    for prong in HLTpath_subdir:
        for eff in HLTpath_eff:
            eff_path = base_dir + '/' + HLT_path + '/' + prong + '/' + eff
            compare_eff(eff_path,
                        file_HIST_ref.Get(eff_path),
                        file_HIST_new.Get(eff_path),
                        'Release 24.0.68',
                        'Release 24.0.72',
                        ymin=0,
                        ymax=1.6
                       )